In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
from IPython.display import HTML, display, clear_output
kgs.set_float32(True)
pack_cuda._ensure_initialized()


In [ ]:
from IPython.display import HTML, display, clear_output
kgs.set_float32(True)
pack_cuda._ensure_initialized()
import pack_cuda_lut
pack_cuda_lut._ensure_initialized()
import lap_batch
lap_batch._ensure_initialized()


In [ ]:
%load_ext pyinstrument

In [ ]:

import pack_ga3
import time
import pack_move
#kgs.enable_module_profiling(pack_move)
runner = pack_ga3.baseline_symmetry_180_tesselated(adapt_moves=True)
runner.ga.ga_base.initializer.ref_sol_axis2_offset = lambda r:0.5
runner.ga.do_legalize = False
runner.ga.ga_base.do_legalize = False
runner.ga.ga_base.N_trees_to_do = 156
runner.n_generations = 10
print(type(runner.rough_relaxers[0].cost.costs[2]))
#runner.rough_relaxers[0].cost.costs[2].use_lookup_table = True
runner.rough_relaxers[0].cost.costs[2]._ensure_lut_initialized()
#runner.fine_relaxers[0].cost.costs[2].use_lookup_table = True
#runner.fine_relaxers[0].cost.costs[2]._ensure_lut_initialized()


In [ ]:
%%pyinstrument
kgs.profiling=True
t = time.time()
runner.run()
print("Elapsed time:", time.time() - t)


In [ ]:
kgs.print_module_profile()
